In [20]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mt-en-vi/src-train.txt
/kaggle/input/mt-en-vi/tgt-val.txt
/kaggle/input/mt-en-vi/src-val.txt
/kaggle/input/mt-en-vi/tgt-train.txt
/kaggle/input/mt-en-vi/src-test.txt
/kaggle/input/mt-en-vi/tgt-test.txt


# Fine-tune mBART50 in translation English to Vietnamese task
> this notebook use IWSLT2015 English-Vietnamese dataset

In [21]:
!pip install transformers
!pip install datasets
!pip install sacrebleu
!pip insall accelerate

ERROR: unknown command "insall" - maybe you meant "install"


In [22]:
from datasets import load_dataset
from transformers import (
    MBartForConditionalGeneration, MBartTokenizer, 
    Seq2SeqTrainingArguments, Seq2SeqTrainer
  )

import torch
from torch.utils.data import random_split
import datasets
import pandas as pd
from transformers import AutoModelForSeq2SeqLM, DataCollatorForSeq2Seq, Seq2SeqTrainingArguments, Seq2SeqTrainer
import numpy as np
from datasets import load_metric
import gc
import torch
from transformers import AutoTokenizer
import datasets

In [23]:
# /kaggle/input/mt-en-vi/src-train.txt
# /kaggle/input/mt-en-vi/tgt-val.txt
# /kaggle/input/mt-en-vi/src-val.txt
# /kaggle/input/mt-en-vi/tgt-train.txt
# /kaggle/input/mt-en-vi/src-test.txt
# /kaggle/input/mt-en-vi/tgt-test.txt
def load_test():
  data = []
  with open("/kaggle/input/mt-en-vi/src-test.txt") as f1, open("/kaggle/input/mt-en-vi/tgt-test.txt") as f2:
      for src, tgt in zip(f1, f2):
        data.append(
            {
                "translation": {
                    "en": src.strip(),
                    "vi": tgt.strip()
                }
            }
        )
  print(f'total size of data is {len(data)}')
  tdata = pd.DataFrame(data)
  tdata = tdata.reset_index()
  tdata = tdata.rename(columns={'index': 'id'})
  test = datasets.Dataset.from_pandas(tdata)
  return test

def load_train():
    data = []
    with open("/kaggle/input/mt-en-vi/src-train.txt") as f1, open("/kaggle/input/mt-en-vi/tgt-train.txt") as f2:
        for src, tgt in zip(f1, f2):
          data.append(
              {
                  "translation": {
                      "en": src.strip(),
                      "vi": tgt.strip()
                  }
              }
          )
    print(f'total size of data is {len(data)}')
    tdata = pd.DataFrame(data)
    tdata = tdata.reset_index()
    tdata = tdata.rename(columns={'index': 'id'})
    dataset = datasets.Dataset.from_pandas(tdata)

    # don't care about test_size. 
    downsampled_dataset = dataset.train_test_split(
        train_size=9000, test_size=500, seed=69
    )
        
    return downsampled_dataset
test_set = load_test()
train_set = load_train()

total size of data is 1268
total size of data is 133317


In [24]:
train_set

DatasetDict({
    train: Dataset({
        features: ['id', 'translation'],
        num_rows: 9000
    })
    test: Dataset({
        features: ['id', 'translation'],
        num_rows: 500
    })
})

In [25]:
checkpoint="facebook/mbart-large-50-many-to-many-mmt"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
print(tokenizer(["Hello, this one sentence!", "This is another sentence."]))

Downloading:   0%|          | 0.00/529 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/649 [00:00<?, ?B/s]

{'input_ids': [[250004, 35378, 4, 903, 1632, 149357, 38, 2], [250004, 3293, 83, 15700, 149357, 5, 2]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [26]:
model = MBartForConditionalGeneration.from_pretrained(checkpoint)

Downloading:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

In [27]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "vi"

def preprocess_function(examples):
    inputs = [ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
    # Setup the tokenizer for targets
    # no need this line 
    # with tokenizer.as_target_tokenizer():
    labels = tokenizer(targets, max_length=max_target_length, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


In [28]:
tokenized_datasets = train_set.map(preprocess_function, batched=True)
tokenized_test_set = test_set.map(preprocess_function, batched=True)

  0%|          | 0/9 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [29]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
metric = load_metric("sacrebleu")

In [30]:

def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels

def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [31]:
from accelerate import Accelerator
accelerator = Accelerator()

In [32]:
# !wandb login --relogin

In [34]:
# release gpu ram
# del trainer

In [35]:
# empty gpu ram
torch.cuda.empty_cache()
gc.collect()

5209

In [36]:
args = Seq2SeqTrainingArguments(output_dir="./mbart_EnglistToVietnamese/",
                        do_train=True,
                        do_eval=True,
                        evaluation_strategy="epoch",
                        per_device_train_batch_size=8,
                        per_device_eval_batch_size=8,
#                         gradient_accumulation_steps=4,
                        learning_rate=5e-5,
                        num_train_epochs=2,
                        predict_with_generate=True,
                        logging_dir="/logs",
                        logging_steps=10000,
                        save_steps=10000,
                        report_to="none"
                        )


trainer = Seq2SeqTrainer(model=model, 
                args=args,
                data_collator=data_collator, 
                train_dataset=tokenized_datasets['train'],
#                eval_dataset=tokenized_datasets['test'],
                 eval_dataset=tokenized_test_set,
                tokenizer=tokenizer,
              compute_metrics=compute_metrics
                )

tokenized_datasets, tokenized_test_set, trainer = accelerator.prepare(
     tokenized_datasets, tokenized_test_set, trainer
      )

trainer.train()
#trainer.evaluate()

The following columns in the training set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 9000
  Num Epochs = 2
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 2250


Epoch,Training Loss,Validation Loss,Bleu,Gen Len
1,No log,1.377274,33.007900,33.500000
2,No log,1.442793,32.582000,33.666400


The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1268
  Batch size = 8
The following columns in the evaluation set don't have a corresponding argument in `MBartForConditionalGeneration.forward` and have been ignored: id, translation. If id, translation are not expected by `MBartForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 1268
  Batch size = 8


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=2250, training_loss=1.0253043619791666, metrics={'train_runtime': 1627.8042, 'train_samples_per_second': 11.058, 'train_steps_per_second': 1.382, 'total_flos': 2266580707442688.0, 'train_loss': 1.0253043619791666, 'epoch': 2.0})

In [37]:
!ls

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
mbart_EnglistToVietnamese


In [38]:
PATH = '/kaggle/working/mbartMT.pth'
torch.save(model.state_dict(), PATH)

In [39]:
%cd /kaggle/working
from IPython.display import FileLink
FileLink('mbartMT.pth')

/kaggle/working


/kaggle/working/mbartMT.pth